<a href="https://colab.research.google.com/github/Marcelodinizsantos/marcelodiniz/blob/main/LEITOR_DE_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Copyright Marcelo Diniz Santos - 2024
# Todos os direitos reservados
# Importar as bibliotecas necessárias!
!pip install -q -U google-generativeai
import ipywidgets as widgets
from IPython.display import display
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('HEMOGRAMA')
genai.configure(api_key=api_key)
import io
!pip install PyPDF2
from PyPDF2 import PdfReader

# Configurar a API key do Google Generative AI


# Função para exibir a mensagem de boas-vindas e permitir o carregamento do PDF
def carregar_pdf():
    # Mensagem de boas-vindas
    print("Seja bem-vindo ao nosso leitor e analista de PDF, carregue o seu documento abaixo.")

    # Widget para carregar o arquivo PDF
    upload_btn = widgets.FileUpload(accept=".pdf", description="Carregar PDF")

    # Widget para enviar o PDF
    send_btn = widgets.Button(description="Enviar")

    # Função para manipular o evento de clique no botão de enviar
    def on_send_btn_clicked(btn):
        if upload_btn.value:
            # Analisar o PDF
            file_name, file_content = next(iter(upload_btn.value.items()))
            file_io = io.BytesIO(file_content["content"])
            pdf_reader = PdfReader(file_io)
            pdf_text = ""
            for page_num in range(len(pdf_reader.pages)):
                pdf_text += pdf_reader.pages[page_num].extract_text()

            # Configurar o modelo Gemini 1.0 Pro
            generation_config = genai.GenerationConfig(temperature=0.5, top_k=40, top_p=0.95, max_output_tokens=256)
            model = genai.GenerativeModel(model_name="gemini-1.0-pro", generation_config=generation_config)

            # Dividir o texto em partes menores
            chunk_size = 500 # Tamanho de cada parte do texto
            text_chunks = [pdf_text[i:i + chunk_size] for i in range(0, len(pdf_text), chunk_size)]

            all_comments = []
            for chunk in text_chunks:
                try:
                  # Solicitar ao Gemini para analisar cada parte
                  response = model.generate_content(f"Analise esta parte do documento e me dê um comentário sobre ela: {chunk}")

                  # Extrair o comentário da resposta
                  comment = response.text
                  all_comments.append(comment)

                except ValueError:
                    print(f".: {chunk}")

            # Combinar os comentários
            final_comment = " ".join(all_comments)

            print(f"Comentário sobre o PDF:\n{final_comment}")

        else:
            print("Por favor, carregue um arquivo PDF primeiro.")

    # Atribuir a função ao evento de clique do botão de enviar
    send_btn.on_click(on_send_btn_clicked)

    # Exibir os widgets
    display(upload_btn)
    display(send_btn)

# Chamada da função para iniciar o processo
carregar_pdf()

Seja bem-vindo ao nosso leitor e analista de PDF, carregue o seu documento abaixo.


FileUpload(value={}, accept='.pdf', description='Carregar PDF')

Button(description='Enviar', style=ButtonStyle())